<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="200" />

<br/>

## Getting Started with the Contextual Platform

Contextual AI lets you create and run specialized AI agents that are powered by your data. This notebook introduces an end-to-end example workflow for creating a Retrieval-Augmented Generation (RAG) agent for a financial use case. The agent will answer questions based on the documents provided, but avoid any forward looking statements, e.g., Tell me about sales in 2028. 

This notebook can be run in under 10 minutes and covers the following steps:
1. Creating a Datastore: this is where your documents live (unstructured data)
2. Ingesting Documents: we will use a single PDF document but the platform is full scalable and supports HTML among other formats
3. Creating an RAG Agent: all you need is a good system prompt to get started
4. Querying an RAG Agent: answers based on your data and nothing more

The full documentation is available at [docs.contextual.ai](https://docs.contextual.ai/)

### Prerequisites:

- API key. You can request one [here](https://contextual.ai/platform/).

- We will share the key with you or you will have to:
 
    1. Login to your account at app.contextual.ai
    2. Click on "API Keys"
    3. Click on "Create API Key". *Note: Please keep your key in a secure place, and do not share it with anyone*

- Python 3.9+

### Environment Setup


1. Clone the repo: `git clone https://github.com/ContextualAI/examples.git`
2. To run this notebook in [VS Code](https://code.visualstudio.com/):

    1.1 Open the terminal, switch to `examples` directory and create a Python virtual environment:

        `python -m venv .`
    1.2 Activate the virtual environment:
        
        `source bin/activate`
    1.3 Intall the dependencies

        `pip install -r requirements.txt`
    1.4 Open the notebook and click on "Select Kernel" (top right) -> "Python Environments" and select the newly created kernel    


In [1]:
import os
from contextual import ContextualAI
from IPython.display import display, Markdown

Replace "your_api_key" with your actual API key 👇🏼

In [2]:
API_KEY = "your_api_key"

In [3]:
os.environ["CONTEXTUAL_API_KEY"] = API_KEY 

contextual = ContextualAI(
    api_key=os.environ.get("CONTEXTUAL_API_KEY"),  # This is the default and can be omitted
)
REQUEST_URL = 'https://api.contextual.ai/v1'

### Step 1: Create your Datastore


You will need to first create a datastore for your agent. A datastore is secure storage for unstructured data (documents). Each agent can have one or more datastores for storing data securely.

In [ ]:
result = contextual.datastores.create(name="financial_research_datastore")
datastore_id = result.id
print(f"Datastore successfully created with ID: {datastore_id}")

### Step 2: Ingest Documents into your Datastore

You can now ingest documents into your Agent's datastore using the /datastores endpoint. Documents must be a PDF or HTML file.


I am using a example PDF. You can also use your own documents here. If you have very long documents (hundreds of pages), processing can take longer.

In [ ]:
with open('intro-end-to-end/data/Apple.pdf', 'rb') as f:
    ingestion_result = contextual.datastores.documents.ingest(datastore_id, file=f)
    document_id = ingestion_result.id
    print(f"Successfully uploaded to datastore document with ID: {datastore_id}")

Once ingested, you can view the list of documents, see their metadata, and also delete documents.

In [ ]:
metadata = contextual.datastores.documents.metadata(datastore_id = datastore_id, document_id = document_id)
print("Document metadata:", metadata)

*Note: make sure the document ingestion status above 👆 shows 'completed' before querying the agent.*

### Step 3: Create your Agent

Next let's create the Agent and modify it to our needs.

Some additional parameters include setting a system prompt or using a previously tuned model.

`system_prompt` is used for the instructions that your RAG system references when generating responses. Note that we do not guarantee that the system will follow these instructions exactly.

In [9]:
system_prompt = '''
You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with numerical values
* Pay attention to fiscal vs. calendar year distinctions
* Present monetary values with appropriate scale (millions/billions)

Response Format:
* Begin with a high-level summary of key findings when analyzing data
* Structure detailed analyses in clear, hierarchical formats
* Use markdown for lists, tables, and emphasized text
* Maintain a professional, analytical tone
* Present quantitative data in consistent formats (e.g., basis points for ratios)

Critical Guidelines:
* Avoid opinions, speculation, or assumptions
* If information is unavailable or irrelevant, clearly state this without additional commentary
* Answer questions directly, then stop
* Do not reference source document names or file types in responses
* Focus only on information that directly answers the query

For any analysis, provide comprehensive insights using all relevant available information while maintaining strict adherence to these guidelines and focusing on delivering clear, actionable information.
'''


Now are ready to create our RAG agent. 

In [ ]:
app_response = contextual.agents.create(
    name="Finacial Research Agent",
    description="Research Agent using only Historical Information",
    system_prompt=system_prompt,
    datastore_ids=[datastore_id]
)
agent_id= app_response.id
print(f"Agent created successfully with ID: {agent_id}")

### Step 4: Query your Agent

Let's query our agent to see if its working and whether the answer provided is correct.

*Note: It may take a few minutes for the document to be ingested and processed. The Assistant will give a detailed answer once the documents are ingested.*

In [ ]:
query_result = contextual.agents.query.create(
    agent_id=agent_id,
    messages=[{
        "content": "what was revenue in for Apple in 2022",
        "role": "user"
    }]
)

display(Markdown(query_result.message.content))

👏🏼 Awesome! In under 10 minutes (hopefully), you were able to create a RAG agent, complete with a scalable datastore and were able to query it.

### Next Steps

 As a next step, check out our [Hands-On Lab] where we show you how to evaluate the accuracy of your RAG agent, edit and tune it to make it more accurate. 
